# Projet 1

In [ ]:
import re

In [26]:
book =  open("LOTR_ReturnOfTheKing.txt").read()
book_chapters = re.split(r"_Chapter \w+_",book)[1:]

In [ ]:
for ChapterNb, Chapter in enumerate(book_chapters):
    
    paragraphsEnsemble = Chapter.split("\n\n")
    
    
    for parag in paragraphsEnsemble: # Per paragraph, check if words are a Character
        